Prerequisites:

`./AppData/Local/Programs/Python/Python310/Scripts/pip install plotly nbformat`

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px

import sys
from pathlib import Path
PROJECT_ROOT = Path().resolve().parents[0]  # notebooks/ → cone-operator-lab/
sys.path.insert(0, str(PROJECT_ROOT))

from modules.hearing_box import scan_A2_grid, Box

In [2]:
path = r"C:/Users/sulta/git/cone-operator-lab/data/eigenvalues/box_a1.0_b1.5_c2.3_N20000.txt"
eigs_20000 = np.loadtxt(path, ndmin=1)
eigs_20000 = np.sort(eigs_20000[eigs_20000 > 0.0])

In [3]:
n_steps = [2000, 5000, 10000]

box = Box(a=1.0, b=1.5, c=2.3)
A2_true = box.A2_exact

In [4]:
# 1) compute once
dfA2 = scan_A2_grid(
    eigs_all=eigs_20000,
    n_steps=n_steps,
    factors=np.arange(10, 120, 1),
    tmax_factor=500,
    m_grid=251,
    diagnostics=False   # diagnostics optional
)

# 2) derive error once (reused everywhere)
dfA2["err_pct"] = np.abs(dfA2["A2_hat"] / A2_true - 1.0) * 100.0

In [5]:
fig = px.line(
    dfA2, x="f", y="err_pct", color="N",
    log_y=True,
    labels={"f": "tMin factor (f/lambda_N)", "err_pct": "relative error of A2 (%)"},
    title="A2 reconstruction error vs f"
)
fig.update_layout(
    width=900,
    height=550,
    margin=dict(l=80, r=40, t=80, b=70)
)
fig.update_traces(mode="lines+markers", hovertemplate="N=%{legendgroup}<br>f=%{x}<br>err=%{y:.4g}%<extra></extra>")
fig.show()

In [6]:
fstar_df = (
    dfA2.loc[dfA2.groupby("N")["err_pct"].idxmin(), ["N", "f", "A2_hat", "err_pct"]]
      .rename(columns={"f": "f_star", "A2_hat": "A2_hat_at_fstar", "err_pct": "rel_error_pct"})
      .sort_values("N")
)

display(fstar_df)

,N,f_star,A2_hat_at_fstar,rel_error_pct
59,2000,69.0,0.121801,0.176998
113,5000,13.0,0.119965,1.332361
225,10000,15.0,0.121313,0.224018
